# Практичкское задание к уроку 7. Сверточные нейронные сети для анализа текста


Берем отызывы за лето (из архива с материалами или предыдущего занятия)

1. Учим conv сеть для классификации

2. Рассмотреть 2-а варианта сеточек 

    2.1. Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/

    2.2. Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

In [1]:
!pip install stop_words

In [2]:
!pip install pymorphy2

In [3]:
!pip install navec

In [4]:
# !pip install numpy
!pip install --upgrade numpy==1.21.0

  Using cached numpy-1.21.0-cp39-cp39-macosx_10_9_x86_64.whl (16.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
tensorflow 2.5.0 requires numpy~=1.19.2, but you have numpy 1.21.0 which is incompatible.
bokeh 2.4.2 requires typing-extensions>=3.10.0, but you have typing-extensions 3.7.4.3 which is incompatible.


In [5]:
!pip install --upgrade pandas==1.2.0

     |████████████████████████████████| 10.7 MB 1.7 MB/s eta 0:00:01
  Attempting uninstall: pandas
    Found existing installation: pandas 1.4.2
    Uninstalling pandas-1.4.2:
      Successfully uninstalled pandas-1.4.2


In [6]:
!pip install gensim

In [7]:
import numpy as np
import pandas as pd
pd.set_option('max_colwidth', 100)
import re
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from navec import Navec
from gensim.models import KeyedVectors

import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/alenakukhta/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
from gensim.models import KeyedVectors

In [9]:
# !pip install google.colab

In [10]:
# conda install -c conda-forge google-colab

In [11]:
!pip install utils

In [12]:
from IPython.utils import traitlets as _traitlets

/var/folders/64/g6jhnpm16ql1vgrrbrbd_yd00000gn/T/ipykernel_22782/3225389896.py:1: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  from IPython.utils import traitlets as _traitlets


In [13]:
# загрузка файла с локального диска
# from google.colab import files
# uploaded = files.upload()

In [14]:
# !pip install xlrd>=1.1.0

In [15]:
df = pd.read_excel('отзывы за лето.xls')
df.head(10)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотят слишком большой доступ к персональным данным в т...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше никуда. Ранее больше года пользовался нормально.,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14
5,5,Всё удобно норм 👍👍👍,2017-08-14
6,5,Очень удобное приложение.,2017-08-14
7,5,Все устраивает,2017-08-14
8,5,"У меня работает все четко. В отличии от банкоматов, которые вечно зависают и тупят",2017-08-14
9,5,Очень все хорошо👍,2017-08-14


#### Предобработка данных

In [16]:
# максимальное количество слов в словаре
max_words = 200
# максимальная длина предложения
max_len = 140
num_classes = 1

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [17]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)
    
df['Content'] = df['Content'].apply(preprocess_text)

In [18]:
df.head(10)

,Rating,Content,Date
0,5,it just works,2017-08-14
1,4,целое удобноной приложениеиз минус хотеть большой доступ персональный данные телефонеприходиться...,2017-08-14
2,5,отлично,2017-08-14
3,5,зависать 1 работа антивирус ранее пользоваться нормально,2017-08-14
4,5,удобно работать быстро,2017-08-14
5,5,удобно норма 👍👍👍,2017-08-14
6,5,удобный приложение,2017-08-14
7,5,устраивать,2017-08-14
8,5,работать чётко отличие банкомат вечно зависать тупить,2017-08-14
9,5,хорошо👍,2017-08-14


In [19]:
df['Rating'].nunique()

5

In [20]:
# Разделим на трейн и тест
df_train, df_test = train_test_split(df, test_size=0.2)
df_test, df_val = train_test_split(df_test, test_size=0.2)
print(f'Train: {df_train.shape}, Test: {df_test.shape}, Val: {df_val.shape}')

Train: (16527, 3), Test: (3305, 3), Val: (827, 3)


In [21]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

In [22]:
def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [23]:
df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_val['Content'] = df_val['Content'].apply(preprocess_text)
df_test['Content'] = df_test['Content'].apply(preprocess_text)

In [24]:
train_corpus = " ".join(df_train["Content"])
train_corpus = train_corpus.lower()

In [25]:
tokens = word_tokenize(train_corpus)

In [26]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [27]:
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [28]:
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный',
 'телефон',
 'супер']

In [29]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [30]:
# переводим текст во последовательность
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    # добавляем нули до размера батча
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [31]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["Content"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["Content"]], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["Content"]], dtype=np.int32)

In [32]:
df_train["Rating"] = df_train["Rating"] - 1
df_val["Rating"] = df_val["Rating"] - 1
df_test["Rating"] = df_test["Rating"] - 1

In [33]:
x_train[1]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0, 49, 43, 22], dtype=int32)

#### Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

In [34]:
# !pip install keras

In [35]:
!pip install utils

In [36]:
# from keras.utils.data_utils import pad_sequences
!pip install keras==2.4.3

In [37]:
!pip install tensorflow==2.5.0

  Using cached numpy-1.19.5-cp39-cp39-macosx_10_9_x86_64.whl (15.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.0
    Uninstalling numpy-1.21.0:
      Successfully uninstalled numpy-1.21.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
bokeh 2.4.2 requires typing-extensions>=3.10.0, but you have typing-extensions 3.7.4.3 which is incompatible.


In [38]:
!pip install dca

In [39]:
# import pkg_resources
# print(f"dca v{pkg_resources.get_distribution('dca').version}")
# print(f"keras v{pkg_resources.get_distribution('keras').version}")

# from dca.api import dca

In [40]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Bidirectional, LSTM, SpatialDropout1D, concatenate
from keras.layers import Dropout
from keras.layers import Activation
from keras.layers import Input
from keras.layers import Embedding
from keras.layers import Conv1D, Flatten, MaxPooling1D
from keras.layers import GlobalMaxPool1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping 

# from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
# import tf.keras.utils.custom_object_scope

In [54]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Bidirectional(LSTM(units=32, return_sequences=True)))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/alenakukhta/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/64/g6jhnpm16ql1vgrrbrbd_yd00000gn/T/ipykernel_22782/3914372017.py", line 4, in <cell line: 4>
    model.add(Bidirectional(LSTM(units=32, return_sequences=True)))
  File "/Users/alenakukhta/opt/anaconda3/lib/python3.9/site-packages/tensorflow/python/training/tracking/base.py", line 522, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "/Users/alenakukhta/opt/anaconda3/lib/python3.9/site-packages/keras/engine/sequential.py", line 223, in add
    output_tensor = layer(self.outputs[0])
  File "/Users/alenakukhta/opt/anaconda3/lib/python3.9/site-packages/keras/layers/wrappers.py", line 581, in __call__
    return super(Bidirectional, self).__call__(inputs, **kwargs)
  File "/Users/alenakukhta/opt/anaconda3/lib/python3.9/site-packages/kera

**Буду благодарна за детальный разбор ошибки. Никак не могу понять что не так делаю.**

In [42]:
num_classes = 5
y_train = tf.keras.utils.to_categorical(df_train["Rating"], num_classes=num_classes)
y_val = tf.keras.utils.to_categorical(df_val["Rating"], num_classes=num_classes)

In [43]:
# model.compile(loss='categorical_crossentropy',
model.compile(loss='pad_sequences',
              optimizer='adam',
              metrics=['accuracy'])

In [44]:
tensorboard = TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping = EarlyStopping(monitor='val_loss')  


# class CustomLayer(keras.layers.Layer):
#     def __init__(self, a):
#         self.var = tf.Variable(a, name="var_a")

#     def call(self, inputs, training=False):
#         if training:
#             return inputs * self.var
#         else:
#             return inputs

#     def get_config(self):
#         return {"a": self.var.numpy()}

#     # There's actually no need to define `from_config` here, since returning
#     # `cls(**config)` is the default behavior.
#     @classmethod
#     def from_config(cls, config):
#         return cls(**config)


# layer = CustomLayer(5)
# layer.var.assign(2)

# serialized_layer = keras.layers.serialize(layer)
# new_layer = keras.layers.deserialize(
#     serialized_layer, custom_objects={"CustomLayer": CustomLayer}
# )

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

2022-06-20 18:54:00.368239: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-06-20 18:54:00.368251: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2022-06-20 18:54:00.368333: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2022-06-20 18:54:00.548414: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20


ValueError: in user code:

    /Users/alenakukhta/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py:830 train_function  *
        return step_function(self, iterator)
    /Users/alenakukhta/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py:813 run_step  *
        outputs = model.train_step(data)
    /Users/alenakukhta/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py:771 train_step  *
        loss = self.compiled_loss(
    /Users/alenakukhta/opt/anaconda3/lib/python3.9/site-packages/keras/engine/compile_utils.py:184 __call__  *
        self.build(y_pred)
    /Users/alenakukhta/opt/anaconda3/lib/python3.9/site-packages/keras/engine/compile_utils.py:133 build  *
        self._losses = tf.nest.map_structure(self._get_loss_object, self._losses)
    /Users/alenakukhta/opt/anaconda3/lib/python3.9/site-packages/tensorflow/python/util/nest.py:867 map_structure  **
        structure[0], [func(*x) for x in entries],
    /Users/alenakukhta/opt/an

In [45]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

ValueError: in user code:

    /Users/alenakukhta/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py:1298 test_function  *
        return step_function(self, iterator)
    /Users/alenakukhta/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py:1282 run_step  *
        outputs = model.test_step(data)
    /Users/alenakukhta/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py:1243 test_step  *
        self.compiled_loss(
    /Users/alenakukhta/opt/anaconda3/lib/python3.9/site-packages/keras/engine/compile_utils.py:184 __call__  *
        self.build(y_pred)
    /Users/alenakukhta/opt/anaconda3/lib/python3.9/site-packages/keras/engine/compile_utils.py:133 build  *
        self._losses = tf.nest.map_structure(self._get_loss_object, self._losses)
    /Users/alenakukhta/opt/anaconda3/lib/python3.9/site-packages/tensorflow/python/util/nest.py:867 map_structure  **
        structure[0], [func(*x) for x in entries],
    /Users/alenakukhta/opt/anaconda3

#### Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/

In [46]:
ru_w2v = KeyedVectors.load_word2vec_format('https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz', binary=True)

In [47]:
def code_w2v_txt(txt, max_len = 100):
    sent_w2v = []
    zero_point = np.zeros(300)
    txt = txt.split()
   
    for i in range(max_len):
        try:
            word = ru_w2v[txt[i]]
        except:
            word = zero_point
        sent_w2v.append(word)
    return np.array(sent_w2v)

max_len = 50
df['w2v'] = df['Content'].apply(code_w2v_txt, max_len = max_len)
del(ru_w2v)

In [48]:
X = list(df['w2v'].values)
X = np.array(X)
y = df['Rating'].values

In [49]:
# Разделим на трейн и тест
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [50]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(y_train) 
test_enc_labels = le.transform(y_test)
le.classes_

array([1, 2, 3, 4, 5])

In [51]:
num_classes = len(df.Rating.unique())

inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))
dropout_embeds = SpatialDropout1D(0.05)(inputs)
x = Bidirectional(LSTM(units=7, return_sequences=True))(dropout_embeds)
pooled_avg_sequences = GlobalAveragePooling1D()(x)
pooled_max_sequences = GlobalMaxPooling1D()(x)
concated = concatenate([pooled_avg_sequences, pooled_max_sequences])
dense_intermediate = Dense(64, activation='elu')(concated)
x = Dense(num_classes, activation='sigmoid')(dense_intermediate)

model = Model(inputs=inputs, outputs=x)
model.summary()
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/alenakukhta/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/64/g6jhnpm16ql1vgrrbrbd_yd00000gn/T/ipykernel_22782/1165963707.py", line 5, in <cell line: 5>
    x = Bidirectional(LSTM(units=7, return_sequences=True))(dropout_embeds)
  File "/Users/alenakukhta/opt/anaconda3/lib/python3.9/site-packages/keras/layers/wrappers.py", line 581, in __call__
    return super(Bidirectional, self).__call__(inputs, **kwargs)
  File "/Users/alenakukhta/opt/anaconda3/lib/python3.9/site-packages/keras/engine/base_layer.py", line 945, in __call__
    return self._functional_construction_call(inputs, args, kwargs,
  File "/Users/alenakukhta/opt/anaconda3/lib/python3.9/site-packages/keras/engine/base_layer.py", line 1083, in _functional_construction_call
    outputs = self._keras_tensor_symbolic_call(
  File "/Users/alenakukhta/opt/a

**Буду благодарна за детальный разбор ошибки. Никак не могу понять что не так делаю.**

In [55]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, train_enc_labels,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

2022-06-20 18:59:07.449371: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-06-20 18:59:07.449386: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2022-06-20 18:59:07.449900: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.


RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [56]:
score = model.evaluate(X_test, test_enc_labels)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [57]:
inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))
l_cov1= Conv1D(128, 5, activation='relu')(inputs)
l_pool1 = MaxPooling1D(2)(l_cov1)
l_cov2 =Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(2)(l_cov2)
l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
l_pool3 = MaxPooling1D(5)(l_cov3)  # global max pooling
l_flat = Flatten()(l_pool3)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(64, activation='softmax')(l_dense)
preds = Dense(num_classes, activation='softmax')(l_dense)

model = Model(inputs=inputs, outputs=preds)

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

In [58]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, train_enc_labels,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

2022-06-20 18:59:50.213178: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-06-20 18:59:50.213191: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2022-06-20 18:59:50.213590: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.


Epoch 1/20
 2/26 [=>............................] - ETA: 3s - loss: 1.6092 - accuracy: 0.2690  

2022-06-20 18:59:55.895907: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-06-20 18:59:55.895923: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2022-06-20 18:59:56.010380: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-06-20 18:59:56.011797: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2022-06-20 18:59:56.020234: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: ./logs/train/plugins/profile/2022_06_20_18_59_56
2022-06-20 18:59:56.020907: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to ./logs/train/plugins/profile/2022_06_20_18_59_56/iMac-Alena.local.trace.json.gz
2022-06-20 18:59:56.023872: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: ./logs/train/plugins/profile/2022_06_20_18_59_56
2022-06-20 18:59:56.0

26/26 [==============================] - 8s 127ms/step - loss: 1.6030 - accuracy: 0.6228 - val_loss: 1.5836 - val_accuracy: 0.7049


In [59]:
score = model.evaluate(X_test, test_enc_labels)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

194/194 [==============================] - 1s 4ms/step - loss: 1.5834 - accuracy: 0.7086


Test score: 1.5834083557128906
Test accuracy: 0.7086156606674194
